<a href="https://colab.research.google.com/github/HelloJocelynLu/EFGs/blob/master/rdkit_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install rdkit-pypi

# Read and write molecules 分子读写篇

## Chemical file format

我们在工作中经常会和化学分子相关的文件打交道，常见的几种文件格式有：
* pdb：通常用于蛋白质
* sdf：通常用于小分子
* xyz
* smi
* 其它：mol, mol2, InchI, InchIKey

由于这次workshop主题主要是针对小分子，因此pdb就不作过多介绍了。辅助小工具：
1. 快速将smiles可视化：https://www.cheminfo.org/flavor/malaria/Utilities/SMILES_generator___checker/index.html
2. 由名字，CAS号或者画图检索分子smiles/sdf等：https://cactus.nci.nih.gov/chemical/structure
3. 图像识别分子结构：https://huggingface.co/spaces/yujieq/MolScribe

我们可以用这个服务得到分子的smiles或者sdf：

In [ ]:
import urllib
def NameToMol(name, ext='smiles'):
    url = "https://cactus.nci.nih.gov/chemical/structure/"+name+"/"+ext
    web = urllib.request.urlopen(url)
    return web.read().decode()

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG=True

import ipywidgets as widgets
from IPython.display import SVG, Image

## Read molecules

In [ ]:
# smiles
smiles = NameToMol('phenol', 'smiles')
mol = Chem.MolFromSmiles(smiles)
mol

**smiles** is not unique! 因此试图用smiles来对比两个分子是不是相同的时候一定要注意。分子中是否包含explicit H（rdkit默认使用implicit H）以及原子是否有mapped number（后面性质介绍会提到）等因素都会影响canonicalize的算法，所以请务必小心。

In [ ]:
for i in range(5):
    print(Chem.MolToSmiles(mol, doRandom=True))

A typical sdf look like:
```java
Protonated structure generated by AutoLigand //Title line (can be blank but line must exist)
 created by AutoLigand @ QuanMol //Program / file timestamp line
//optional comments
  6  5  0  0  0  0  0  0  0  0999 V2000
   -0.3719    0.0024    0.0028 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8980   -0.5748   -0.1191 O   0  0  0  0  0  0  0  0  0  0  0  0
   -0.6741   -0.1194    1.0508 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.3142    1.0665   -0.3155 H   0  0  0  0  0  0  0  0  0  0  0  0
   -1.0830   -0.5246   -0.6430 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.5452    0.1499    0.0240 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0
  1  3  1  0
  1  4  1  0
  1  5  1  0
  2  6  1  0
M  END
$$$$
```

In [ ]:
with open("sample.sdf", "w") as wf:
    print(NameToMol('phenol', 'sdf'), file=wf)

In [ ]:
mol = Chem.SDMolSupplier("sample.sdf")[0]
mol

mol, mol2 文件也属于MolBlockFile的类型，形式和sdf很像，可以用`Chem.MolFromMolFile(name)`的方式读入。xyz文件则非常简洁：
```
11

C       -0.180226841      0.360945118     -1.120304970
C       -0.180226841      1.559292118     -0.407860970
C       -0.180226841      1.503191118      0.986935030
N       -0.180226841      0.360945118      1.29018350
...
```
因为它不包含键连信息，而在没有smiles的情况下，仅根据坐标推测键连信息效果不佳，不建议直接用rdkit与xyz文件交互（除非在有template，或是不在乎键连信息的情况下）

## Write molecules

rdkit有若干方法可以把`mol`对象输出，下面给出了将分子转化为smiles/xyz/mol/inchi/inchikey的方法。和smiles不一样的是，一个分子的inchi(key)通常是唯一的。

In [ ]:
print_params={
    "SMILES:": Chem.MolToSmiles,
    "XYZ:": Chem.MolToXYZBlock,
    "MOlBlock:": Chem.MolToMolBlock,
    "InChI:": Chem.MolToInchi,
    "InChIKey:": Chem.MolToInchiKey,
}
for k,v in print_params.items():
    print(k)
    print(v(mol))
    print('-'*75)

除了直接输出文本，也可以输出到文件。我们最常用的sdf就可以这样保存：

In [ ]:
writer = Chem.SDWriter("new.sdf")
writer.write(mol)
writer.close()

值得一题的是，当分子不能被rdkit识别的时候，会返回`None`。因此大批量处理分子的时候，可以加入一个判断：
```python
for ... in ...:
    mol = ...
    if not mol:
        continue
    ... do something ...
```

In [ ]:
mol = Chem.MolFromSmiles("ThisIsBad")
assert mol is None

## 练习

得到分子`aspirin`的smiles和sdf，并回答下面的问题。smiles和sdf都可以从上面`NameToMol`的函数得到，但作为练习目的，请按这样的流程操作：

    1. 用`NameToMol`得到分子的smiles
    2. 用分子的smiles创建rdkit的mol对象
    3. 将这个mol对象存为sdf

问题：
1. 这个分子里有几个重原子？
2. 这个分子的inchikey里面前五位是什么？
3. 这个分子canonical smiles是什么？

思考：

4. 用可视化软件或者文本打开存下来的sdf，有没有发现什么值得注意的地方？

In [ ]:
# code here
smiles = NameToMol("aspirin")
mol = Chem.MolFromSmiles(smiles)

writer = Chem.SDWriter("new.sdf")
writer.write(mol)
writer.close()

print(mol.GetNumHeavyAtoms())
print(mol.GetNumAtoms())
print(Chem.MolToInchiKey(mol))
print(Chem.MolToSmiles(mol))

In [ ]:
# @title 请填写或选择正确的答案，然后运行代码 { display-mode: "form" }

# @markdown ### 这个分子里有几个重原子？
num_atom = "0" # @param [0, 11, 13, 15, 17, 19, 21]

# @markdown ### 这个分子的inchikey里面前五位是什么？
inchi_key = "" # @param {type:"string"}

# @markdown ### 这个分子canonical smiles是什么？
smiles = "" # @param {type:"string"}

if num_atom == "13" and inchi_key == "BSYNR" and smiles == "CC(=O)Oc1ccccc1C(=O)O":
  print("全部答案正确！")
else:
  print("不对哦，再试试吧！")

# 分子性质篇

## 分子的性质

分子有一些非常基础的性质，如有多少颗原子、多少根键、多少构象等等。

In [ ]:
print(mol.GetNumAtoms(), mol.GetNumBonds(), mol.GetNumConformers())

但除了这几个基础性质以外，还有很多其他的性质，一些我们熟悉的性质包括：
* MolWt
* qed
* MolLogP
* RingCount
* NumRotatableBonds
* NumHAcceptors
* NumHDonors
* NumAromaticRings
* HeavyAtomCount
* ...

我们可以直接利用这些关键字，计算并获取相应的性质：

In [ ]:
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
descriptor_names = ['MolWt', 'qed', 'MolLogP', 'RingCount', 'NumRotatableBonds', 'NumHDonors', 'NumHAcceptors']
calculator = MolecularDescriptorCalculator(descriptor_names)
descriptors = calculator.CalcDescriptors(mol)
descriptors

完整的描述符列表可以在这里找到：

In [ ]:
from rdkit.Chem import Descriptors
Descriptors._descList

分子指纹molecular fingerprint算是另一个比较常用的分子层面描述符。rdkit提供了很多相关计算方法，这里仅以Morgan fingerprint的默认参数为例，看看分子指纹是怎么用的：

In [ ]:
mols = ['Cc1ccccc1', 'Cc1ncccc1', 'CC(=O)Oc1ccccc1C(=O)O']
for m in mols:
    print(m)
    display(Chem.MolFromSmiles(m))

In [ ]:
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
m1 = Chem.MolFromSmiles(mols[0])
fp1 = AllChem.GetMorganFingerprint(m1,2)
m2 = Chem.MolFromSmiles(mols[1])
fp2 = AllChem.GetMorganFingerprint(m2,2)
m3 = Chem.MolFromSmiles(mols[2])
fp3 = AllChem.GetMorganFingerprint(m3,2)
print("Similarity between 0 and 1:", DataStructs.DiceSimilarity(fp1,fp2))
print("Similarity between 0 and 2:", DataStructs.DiceSimilarity(fp1,fp3))

所以我们看到，第一个分子和第二个分子 比 第一个分子和第三个分子 更相似，这也符合我们自己的观察。

除了rdkit支持的性质，我们也可以给分子我们自定义的性质：

In [ ]:
mol.SetProp("Label", "Best molecule")
mol.GetPropsAsDict()

## 原子的性质

得到分子中的一个原子，最常用的方法：
* 遍历分子中的所有原子：`mol.GetAtoms()`
* 按指定原子索引获取原子：`mol.GetAtomWithIdx(x)`

原子有很多性质可以获取，很多性质都能容易从名字中看出来：

In [ ]:
for atom in mol.GetAtoms():
    print(atom.GetIdx(), atom.GetAtomicNum(), atom.GetSymbol(), atom.GetFormalCharge(), atom.GetDegree(), atom.GetChiralTag())

In [ ]:
[x for x in dir(atom) if not x.startswith('_')]  # 一系列可用的方法

原子还能和其他原子以及键进行交互，比方说得到相邻的原子(`atom.GetNeighbors()`)或者键(`atom.GetBonds()`)，这为很多操作提供了可能。

和分子类似，原子也可以自定义属性，有的保留关键字还会有自己的作用。比方说下面这个例子就可以把原子索引标在原子旁边：

In [ ]:
for atom in mol.GetAtoms():
    atom.SetProp("atomNote", str(atom.GetIdx()))
display(mol)

## 键的性质

得到分子中的一根键，最常用的方法：
* 遍历分子中的所有键：`mol.GetBonds()`
* 按指定键索引获取键：`mol.GetBondWithIdx(x)`
* 根据起始原子索引获取键：`mol.GetBondBetweenAtoms(a1, a2)`

键也有很多性质可以获取，很多性质都能容易从名字中看出来：

In [ ]:
# 试着获取原子10，12之间的双键
bond1 = mol.GetBondBetweenAtoms(10, 12)
bond2 = mol.GetBondBetweenAtoms(12, 10)
print(bond1.GetIdx(), bond2.GetIdx())
print(bond1, bond2)
print(bond1.GetBeginAtomIdx(), bond2.GetBeginAtomIdx())
print(bond1.GetBondTypeAsDouble(), bond2.GetBondTypeAsDouble())

这边可以看出来原子前后顺序没关系，都可以得到同样index的键。这两个键指针位置不一样，但是属性会是一样的。注意一下，当两个原子之间没有化学键相连的时候，`GetBondBetweenAtoms`会返回None。类似前面的性质，键的属性也可以自定义，但这里不再赘述。

## 练习

对于一个smiles为`Nc1nccc2scc(-c3ccc4c(c3)CCN4C(=O)Cc3ccccc3)c12`的分子，回答如下问题：
1. 这个分子的分子质量是多少？
2. 这个分子的第6个原子元素符号是什么？
3. 这个分子里有哪些原子是非芳香的（提示：属性`atom.GetIsAromatic()`为False)
4. 在非芳香的原子中，是否存在两个原子是用双键连接的，这两个原子索引分别是什么？

In [ ]:
# code here
new_mol = Chem.MolFromSmiles("Nc1nccc2scc(-c3ccc4c(c3)CCN4C(=O)Cc3ccccc3)c12")
print(Descriptors.MolWt(new_mol))
print(new_mol.GetAtomWithIdx(6).GetSymbol())
non_aromatic = [atom.GetIdx() for atom in new_mol.GetAtoms() if not atom.GetIsAromatic()]
print(non_aromatic)
for bond in new_mol.GetBonds():
    if bond.GetBondTypeAsDouble() == 2 and bond.GetBeginAtomIdx() in non_aromatic and bond.GetEndAtomIdx() in non_aromatic:
        print(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())

In [ ]:
# @title 请填写或选择正确的答案，然后运行代码 { display-mode: "form" }

# @markdown ### 这个分子的分子质量是多少？(四舍五入取整)
mol_wt = 0 # @param {type:"slider", min:0, max:400, step:1}

# @markdown ### 这个分子的第6个原子元素符号是什么？
element = "C" # @param ["C", "H", "O", "N", "S"]

# @markdown ### 这个分子里有哪些原子是非芳香的，请用空格隔开（提示：属性atom.GetIsAromatic()为False)
non_aromatic = "" # @param {type:"string"}

# @markdown ### 在非芳香的原子中，是否存在两个原子是用双键连接的，这两个原子索引分别是什么？请用空格隔开
atom_idx = "" # @param {type:"string"}

if mol_wt == 385 and element == "S" and tuple(sorted(non_aromatic.split())) == ("0", "15", "16", "17", "18", "19", "20") and tuple(sorted(atom_idx.split())) == ("18", "19"):
  print("全部答案正确！")
else:
  print("不对哦，再试试吧！")

# 分子结构篇

## 基础结构编辑

简单的结构编辑可以仅用Chem.rdmol.Mol实现，最常用的编辑是使用`AddHs`给分子加上H。默认参数下，无论是从sdf还是smiles读分子，都用的隐式H模型，因此如果要对H原子进行操作，则需要给分子加上显式H：

In [ ]:
mol = Chem.MolFromSmiles("Nc1nccc2scc(-c3ccc4c(c3)CCN4C(=O)Cc3ccccc3)c12")
mol_with_H = Chem.AddHs(mol)
print(Chem.MolToSmiles(mol), mol.GetNumAtoms(), "atoms")
display(mol)
print(Chem.MolToSmiles(mol_with_H), mol_with_H.GetNumAtoms(), "atoms")
display(mol_with_H)

注意，AddHs本身不会改变分子的质子化状态，只是忠实的按照价键补上H而已。

我们也能对分子结构的一部分进行删改操作：

In [ ]:
print("Replace the fused heterocyclic ring with *")
replaced_mol = Chem.ReplaceCore(mol, Chem.MolFromSmiles('C1CC2=C(N1)C=CC=C2'))
replaced_mol.UpdatePropertyCache()
display(replaced_mol)
print("Replace the side chain of fused heterocyclic with *:")
replaced_mol = Chem.ReplaceSidechains(mol, Chem.MolFromSmiles('C1CC2=C(N1)C=CC=C2'))
replaced_mol.UpdatePropertyCache()
display(replaced_mol)
print("Replace one of the H with F:")
replaced_mol = Chem.ReplaceSubstructs(mol_with_H, Chem.MolFromSmarts("[H]"), Chem.MolFromSmiles("F"))[0]
replaced_mol.UpdatePropertyCache()
display(replaced_mol)
print("Replace all of the Hs with Fs:")
replaced_mol = Chem.ReplaceSubstructs(mol_with_H, Chem.MolFromSmarts("[H]"), Chem.MolFromSmiles("F"), replaceAll=True)[0]
replaced_mol.UpdatePropertyCache()
display(replaced_mol)

在rdkit中，还有提供对分子结构更为精细的编辑方法:我们可以将分子创建为一个RWMol的对象，然后指定原子或者键的索引进行替换。
RWMol类提供了`AddAtom`, `AddBond`, `RemoveAtom`, `RemoveBond`, `ReplaceAtom`, `ReplaceBond`等方法。

In [ ]:
rwmol = Chem.RWMol(mol_with_H)
rwmol.ReplaceAtom(28, Chem.Atom('F'))
rwmol.UpdatePropertyCache()
display(rwmol.GetMol())

## 结构匹配

你可能会注意到，刚才我们看到在结构替换中，我们用到了`Chem.MolFromSmarts("[H]")`。SMARTS (SMiles ARbitrary Target Specification)也是一种分子的线性表示。但它允许定义一些分子结构的模式。这使它在结构搜索和匹配中比SMILES更有用。SMARTS allows for the specification of substructures or patterns within a molecule, making it useful for searching and matching chemical structures.

下面的例子试图找到两个分子之间的最大公有子结构：

In [ ]:
from rdkit.Chem import rdFMCS
mol1 = Chem.MolFromSmiles("O=C(NCc1cc(OC)c(O)cc1)CCCCC=CC(C)C")
mol2 = Chem.MolFromSmiles("CC(C)C(=O)NCC1=CC(=C(C=C1)O)OC")
res = rdFMCS.FindMCS([mol1, mol2])
display(mol1), display(mol2)

In [ ]:
res.numAtoms, res.smartsString

我们可以从smarts出发，创建对应的smarts结构。[这里](https://smarts.plus/)可以将一个smarts可视化。

In [ ]:
mcs_smarts = res.smartsString
pattern = Chem.MolFromSmarts(mcs_smarts)
mol1.GetSubstructMatch(pattern)
mol2.GetSubstructMatch(pattern)
display(mol1), display(mol2)

结合分子编辑和结构匹配，我们能做很多事情，比方说想要将分子中非芳香且非sp2的N设为+1价，我们可以：

In [ ]:
def protonN(mol):
    patt = Chem.MolFromSmarts("[#7&!a&!^2]")
    matches = mol.GetSubstructMatches(patt)
    for m in matches:
        for atom_idx in m:
            atom = mol.GetAtomWithIdx(atom_idx)
            atom.SetFormalCharge(1)
    return mol

In [ ]:
from rdkit.Chem import Draw
# list of SMILES
smiList = ['c1ccc2[nH]ccc2c1',
       'Nc1ccccc1',
       'NCc1cccnc1',
       'NCNC(=O)c1ccccc1CN',
       'C[N+](C)(C)CCC([O-])=O']

# Create RDKit molecular objects
mols = [protonN(Chem.MolFromSmiles(m)) for m in smiList]

# display
Draw.MolsToGridImage(mols,molsPerRow=3,subImgSize=(200,200))

## 3D分子

上面我们提到的都是分子在平面的编辑。但现实世界的分子是有着3维结构的。rdkit提供了生成三维结构的方法`EmbedMolecule`，在生成结构的时候，建议给分子加上显式H，这样能使结构生成得更好一点。在没生成3维结构的时候，我们能看到分子的z坐标都是0：

In [ ]:
mol = Chem.MolFromSmiles("C")
mol_with_H = Chem.AddHs(mol)
print(Chem.MolToMolBlock(mol_with_H))

生成3维结构了以后：

In [ ]:
AllChem.EmbedMolecule(mol_with_H)
print(Chem.MolToMolBlock(mol_with_H))

感兴趣的同学可以之后把这两个分子存下来，然后用可视化软件看看这两个结构长什么样子。

## 练习

对于smiles为`Oc1ccccc1`的分子，把它：
1. 将O换成C（OH -> CH3)，并写出替换以后分子的smiles
2. 得到它和分子`Nc1ccccc1`最大公有子结构的smarts字符串

In [ ]:
# code here
mol = Chem.MolFromSmiles("Oc1ccccc1")
replaced_mol = Chem.ReplaceSubstructs(mol, Chem.MolFromSmarts("[O]"), Chem.MolFromSmiles("C"), replaceAll=True)[0]
replaced_mol.UpdatePropertyCache()
print(Chem.MolToSmiles(replaced_mol))

new_mol = Chem.MolFromSmiles("Nc1ccccc1")
res = rdFMCS.FindMCS([mol, new_mol])
print(res.smartsString)

In [ ]:
# @title 请填写或选择正确的答案，然后运行代码 { display-mode: "form" }

# @markdown ### 将O换成C（OH -> CH3)，并写出替换以后分子的smiles
smiles = "" # @param {type:"string"}

# @markdown ### 得到它和分子Nc1ccccc1最大公有子结构的smarts字符串
mcs_smarts = "" # @param {type:"string"}

if smiles == "Cc1ccccc1" and mcs_smarts == "[#6]1:[#6]:[#6]:[#6]:[#6]:[#6]:1":
  print("全部答案正确！")
else:
  print("不对哦，再试试吧！")

# 分子可视化及其它

## 高亮原子/键

在做演示的时候，有时候会需要展示分子结构。这时可能会有一些高亮部分子结构的需求，可能你会注意到，在我们之前调用`GetSubstructMatch`的时候，MCS已经被高亮了。除此以外，rdkit里面Draw提供了一些方法可以高亮原子或者键。

贯穿本次workshop的例子里其实已经有将分子inline的画在jupyter notebook的例子了，接下来我们将一起看怎么直接将分子存成图片格式。在这个例子里，我们需要在`MolDraw2DCairo`中传入分子图片的长宽，然后在`DrawMolecule`传入参数`highlightAtoms`，是需要被高亮的原子索引的列表。默认来说，如果一根键两端的原子都被高亮了，那根键也会被高亮。这个行为可以通过`highlightBonds`干预（如联苯中，想要分别高亮两个苯环，但不想高亮中间连着的那根键）

In [ ]:
# 将图片存为png格式
from rdkit.Chem.Draw import rdMolDraw2D
mol = Chem.MolFromSmiles("Nc1nccc2scc(-c3ccc4c(c3)CCN4C(=O)Cc3ccccc3)c12")
drawer = rdMolDraw2D.MolDraw2DCairo(300, 300) # image size
drawer.DrawMolecule(mol, highlightAtoms=[9,10,11,12,13,14,15,16,17,18,19])
drawer.FinishDrawing()
drawer.WriteDrawingText("sample_mol.png")

In [ ]:
# @title 快速预览分子的原子索引 { display-mode: "form" }


# @markdown ### 请输入分子的smiles
smiles = "Nc1nccc2scc(-c3ccc4c(c3)CCN4C(=O)Cc3ccccc3)c12" # @param {type:"string"}
mol = Chem.MolFromSmiles(smiles)

for atom in mol.GetAtoms():
    atom.SetProp("atomNote", str(atom.GetIdx()))

print("Preview:")
display(mol)

In [ ]:
# @title  { display-mode: "form" }
# @markdown ### 请选择你想高亮的原子编号：
selected = widgets.SelectMultiple(
    options=[i for i in range(mol.GetNumAtoms())],
    value=[9,10,11,12,13,14,15,16,17,18,19],
    rows=10,
    description='Atom index',
    disabled=False
)

display(selected)
button = widgets.Button(description="Show!")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  highlightAtoms = list(selected.value)
  drawer = rdMolDraw2D.MolDraw2DSVG(300, 300) # image size
  drawer.DrawMolecule(mol, highlightAtoms=highlightAtoms)
  drawer.FinishDrawing()
  svg = drawer.GetDrawingText()
  display(SVG(svg.replace('svg:','')))

button.on_click(on_button_clicked)
display(button, output)

我们也可以给高亮部分染上不同的颜色，甚至是根据性质染色。下面给出一个例子，将原子按原子序号染色：

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
cmap = plt.get_cmap('cool')
norm = Normalize(vmin=5, vmax=9)
atom_colors = {}
for atom in mol.GetAtoms():
    atom_colors[atom.GetIdx()] = cmap(norm(atom.GetAtomicNum()))[:3]
# 将图片存为svg格式
drawer = rdMolDraw2D.MolDraw2DSVG(300, 300) # image size
drawer.DrawMolecule(mol, highlightAtoms=atom_colors.keys(), highlightBonds=[], highlightAtomColors=atom_colors)
drawer.FinishDrawing()
svg = drawer.GetDrawingText()
with open("sample_mol.svg", 'w') as wf:
    print(svg, file=wf)

## 批量显示分子

`MolsToGridImage`这个方法可用于批量显示分子，其中`molsPerRow`可以控制一行显示几个，`maxMols`可控制最多显示几个（默认是50）

In [ ]:
from copy import deepcopy
mol = Chem.MolFromSmiles("c1ccc2ccccc2c1")
mols, legends = [], []
for atom in mol.GetAtoms():
    if atom.GetDegree() < 3:
        new_mol = Chem.RWMol(mol)
        new_mol.ReplaceAtom(atom.GetIdx(), Chem.Atom("N"))
        Chem.SanitizeMol(new_mol)
        mols.append(new_mol.GetMol())
        legends.append("Replace # "+str(atom.GetIdx())+" atom!")
Draw.MolsToGridImage(mols, molsPerRow=4, legends=legends)

## 打包分子

在和同学讨论交流的时候，如果在讨论bug，有的涉及rdkit分子相关的问题不好复现，这个时候可以将Mol对象pickle起来：

In [ ]:
import pickle
prob_mol = Chem.MolFromSmiles('c1cccc1', sanitize=False)
prob_mol

In [ ]:
# 打包
with open("error_mol.pkl", 'wb') as f:
    pickle.dump(prob_mol, f)
# 拆包
with open("error_mol.pkl", 'rb') as f:
    unpacked_mol = pickle.load(f)
unpacked_mol

## 其他资源

rdkit这个包其实有很多好用的方法，但很多时候文档并不是很详细。很多的需求即使没在文档里，也都被实现过或者被讨论过了。可以在[github](https://github.com/rdkit/rdkit)页面检索issue或者discussion（甚至提交一个自己的问题）。搜索引擎出来的文档页面有时候也会比较混乱，这时我们也可以直接查看相关函数的docstring：

In [ ]:
?Chem.AddHs